## Daily feature pipeline

In [1]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)
# Set the environment variables from the file <root_dir>/.env
from backend.models import config
settings = config.HopsworksSettings(_env_file=f"{project_root}/.env")

HopsworksSettings initialized!


In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
from backend import util
import datetime
from pathlib import Path
import json
import re
import great_expectations as ge
import warnings
warnings.filterwarnings("ignore")

### Initialize project

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 


2025-11-17 15:57:59,579 INFO: Initializing external client
2025-11-17 15:57:59,579 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 15:58:01,184 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279162


Initialize parameters, parameters for papermill otherwise from .env.

In [4]:
today = datetime.date.today()

csv_file=f"{project_root}/backend/data/rahu-air-quality.csv"

util.check_file_path(csv_file)

# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value() 
country = locals().get("country", "sweden")
city = locals().get("city", "stockholm")
# latitude, longitude = util.get_city_coordinates("stockholm")
latitude, longitude = 59.33, 18.07
print(latitude, longitude)

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

File successfully found at the path: /Users/jonaslorenz/Desktop/Code_KTH/ScalableML/AirQualityPrediction/backend/data/rahu-air-quality.csv
59.33 18.07
Found AQICN_API_KEY: 85a05e409148fc36fa8a5aef727bc004a02fa69c
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

Get back the feature groups

In [5]:
air_quality_fg = fs.get_feature_group(
    name='air_quality_with_all_lags',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

Get today's air quality data.

In [6]:
import yaml
with open(f"{project_root}/backend/sensors/sensors.yml", "r") as f:
        config = yaml.safe_load(f)

today_list = []

for sensor in config["sensors"]:
    
    today_list.append(
        util.get_pm25(
            sensor['aqicn_url'], 
            sensor['country'], 
            sensor['city'], 
            sensor['street'], 
            today, 
            AQICN_API_KEY
        )
    )

aq_today_df = pd.DataFrame(today_list)
aq_today_df

,pm25,country,city,street,date,url
0,20.0,sweden,stockholm,stockholm-st-eriksgatan-83,2025-11-17,https://api.waqi.info/feed/@10523
1,13.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2025-11-17,https://api.waqi.info/feed/@10009
2,5.0,sweden,stockholm,hammarby,2025-11-17,https://api.waqi.info/feed/A63637
3,1.0,sweden,stockholm,ljusslingan,2025-11-17,https://api.waqi.info/feed/A65272
4,0.0,sweden,stockholm,sweden-hägersten-älvsjö-stadsdelsområde-nyboho...,2025-11-17,https://api.waqi.info/feed/A474841
5,9.0,sweden,stockholm,sollentuna-sollentunavagen-192,2025-11-17,https://api.waqi.info/feed/@13984
6,0.0,sweden,stockholm,liljeholmen,2025-11-17,https://api.waqi.info/feed/A229291
7,12.0,sweden,stockholm,sundbyberg-tulegatan-9,2025-11-17,https://api.waqi.info/feed/@13988
8,3.0,sweden,stockholm,jultomtestegen,2025-11-17,https://api.waqi.info/feed/A78529
9,3.0,sweden,stockholm,nacka,2025-11-17,https://api.waqi.info/feed/A196735


Get weather forecast data.

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-17,-0.45,0.0,10.948973,316.332153,stockholm
1,2025-11-18,-0.75,0.0,8.913181,313.363464,stockholm
2,2025-11-19,-0.50,0.9,8.121970,12.804273,stockholm
3,2025-11-20,0.50,0.0,14.264361,10.175456,stockholm
4,2025-11-21,-3.95,0.0,12.287555,301.827393,stockholm
5,2025-11-22,1.75,0.0,11.631956,248.198532,stockholm
6,2025-11-23,2.10,0.0,10.464798,86.054893,stockholm


Compute lag for the last three days and upload today's air quality data into the air quality feature group.

In [8]:
aq_all = air_quality_fg.read()
aq_all["date"] = pd.to_datetime(aq_all["date"]).dt.tz_localize(None)

aq_loc = aq_all[
    (aq_all["country"] == country) &
    (aq_all["city"] == city)
].copy()

aq_all = aq_all.sort_values("date").reset_index(drop=True)

lag_feature_list = []

for sensor in config["sensors"]:
    
    lag_feature_list.append(
        util.compute_lag_features(
            aq_all,
            sensor['country'], 
            sensor['city'], 
            sensor['street']
        )
    )

lag_feature_frame = pd.DataFrame(lag_feature_list)
print(len(lag_feature_list))
df_to_insert = aq_today_df.merge(lag_feature_frame, on=['country', 'city', 'street'])
df_to_insert

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.86s) 
11


,pm25,country,city,street,date,url,pm25_lag_1,pm25_lag_2,pm25_lag_3,pm25_rolling_2d,pm25_rolling_3d
0,20.0,sweden,stockholm,stockholm-st-eriksgatan-83,2025-11-17,https://api.waqi.info/feed/@10523,11.0,4.0,8.00,7.5,7.666667
1,13.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2025-11-17,https://api.waqi.info/feed/@10009,5.0,6.0,8.00,5.5,6.333333
2,5.0,sweden,stockholm,hammarby,2025-11-17,https://api.waqi.info/feed/A63637,4.0,4.0,1.48,4.0,3.160000
3,1.0,sweden,stockholm,ljusslingan,2025-11-17,https://api.waqi.info/feed/A65272,1.0,1.0,0.20,1.0,0.733333
4,0.0,sweden,stockholm,sweden-hägersten-älvsjö-stadsdelsområde-nyboho...,2025-11-17,https://api.waqi.info/feed/A474841,0.0,0.0,0.00,0.0,0.000000
5,9.0,sweden,stockholm,sollentuna-sollentunavagen-192,2025-11-17,https://api.waqi.info/feed/@13984,5.0,5.0,7.00,5.0,5.666667
6,0.0,sweden,stockholm,liljeholmen,2025-11-17,https://api.waqi.info/feed/A229291,0.0,0.0,2.00,0.0,0.666667
7,12.0,sweden,stockholm,sundbyberg-tulegatan-9,2025-11-17,https://api.waqi.info/feed/@13988,9.0,14.0,12.00,11.5,11.666667
8,3.0,sweden,stockholm,jultomtestegen,2025-11-17,https://api.waqi.info/feed/A78529,3.0,3.0,1.05,3.0,2.350000
9,3.0,sweden,stockholm,nacka,2025-11-17,https://api.waqi.info/feed/A196735,2.0,2.0,0.90,2.0,1.633333


In [11]:
df_to_insert['pm25'] = df_to_insert['pm25'].astype('float32')
air_quality_fg.insert(df_to_insert)

2025-11-17 15:59:26,670 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1717611


Uploading Dataframe: 100.00% |██████████| Rows 11/11 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_with_all_lags_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/air_quality_with_all_lags_1_offline_fg_materialization/executions


(Job('air_quality_with_all_lags_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_rolling_2d",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 757786
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 11,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T02:59:26.000670Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

Insert today's weather.

In [12]:
weather_fg.insert(daily_df, wait=True)

2025-11-17 15:59:40,292 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1638074


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-17 15:59:57,044 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 16:00:00,458 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 16:02:06,135 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 16:02:06,285 INFO: Waiting for log aggregation to finish.
2025-11-17 16:02:18,678 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735353
         }
       },
       "result": {
         "observed_value": 8.121970176696777,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T02:59:40.000292Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 